In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql

# host, user, password, db 정보 입력하기
db = pymysql.connect(host='', port=3306, user='', password='', db='', charset='utf8')

cursor = db.cursor()

def get_books(genre):
    for i in range(genre[2], genre[3]+1):
        url = f"http://www.yes24.com/24/Category/Display/{genre[1]}?PageNumber={i}"
        request = requests.get(url)
        soup = BeautifulSoup(request.text, "html.parser")

        books = soup.select("li div.cCont_goodsSet")
        for book in books:
            try:
                img = book.select("p.goods_img img")[0]["src"]
            except IndexError:
                img = ""
            book_code = book.select("div.goods_name a")[0]["href"]
            book_url = f"http://www.yes24.com{book_code}"
            title = book.select("div.goods_name a")[0].text.replace("'","")
            try:
                author = book.select("span.goods_auth a")[0].text.replace("'","")
            except IndexError:
                author = ""
            try:
                pub = book.select("span.goods_pub")[0].text.replace("'","")
            except IndexError:
                pub = ""
            try:
                date = book.select("span.goods_date")[0].text.replace("년 ",".").replace("월","")
            except IndexError:
                date = ""
            try:
                price = book.select("div.goods_price em")[0].text.replace(",","")
            except IndexError:
                price = ""
            try:
                review = book.select("div.goods_rating span.gd_reviewCount em")[0].text.replace("'","")
                rating = book.select("div.goods_rating span.gd_rating em")[0].text
            except IndexError:
                review = "0"
                rating = "0"
            try:
                story = book.select("div.goods_read")[0].text.lstrip().rstrip().replace("'","")
            except IndexError:
                story =""
    
            book = {
                "genre":genre[0],
                "img" : img,
                "url" : book_url,
                "title" : title,
                "author" : author,
                "pub" : pub,
                "date" : date,
                "price" : price,
                "review" : review,
                "rating" : rating,
                "story" : story,
            }
            #print(book)
            
            # 넣고자 하는 테이블 명 입력하기
            sql = """INSERT INTO 테이블명 (title, author, pub, date, price, review, rating, story, img, url, genre) VALUES(
            '""" + book['title'] + """',
            '""" + book['author'] + """', 
            '""" + book['pub'] + """', 
            '""" + book['date'] + """',
            '""" + book['price'] + """',
            '""" + book['review'] + """',
            '""" + book['rating'] + """',
            '""" + book['story'] + """',
            '""" + book['img'] + """',
            '""" + book['url'] + """',
            '""" + book['genre'] + """')"""
            sql.replace('\n', '')
            cursor.execute(sql)

mystery = ["추리/미스터리", "001001046011001", 1, 208]
horror = ["공포/스릴러", "001001046011002", 1, 163]
fantasy = ["판타지", "001001046011003", 1, 1857]
martial = ["무협", "001001046011004", 1, 727]
sf = ["SF", "001001046011005", 1, 69]
history = ["역사", "001001046011006", 1, 227]
romance = ["로맨스", "001001046011007", 1, 742]

genres = [mystery, horror, fantasy, martial, sf, history, romance]
for genre in genres:
    get_books(genre)
    
db.commit()

db.close()